In [35]:
import sys
import os
import pandas as pd

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# Get the current working directory
# Get project root (folder above 'notebooks')
project_root = os.path.dirname(os.getcwd())

# Path to scripts folder
scripts_path = os.path.join(project_root, "scripts")

# Add to sys.path
sys.path.append(project_root)
sys.path.append(scripts_path)

In [36]:
from utils import drop_unusable_column,convert_object_to_numeric,impute_missing_values,create_vehicle_age_feature

In [37]:
df = pd.read_csv(
    '../data/MachineLearningRating_v3.txt',
    sep='|',
    encoding='utf-8',
    engine='python'
)

In [38]:
df_clean = drop_unusable_column(df)
df_clean = convert_object_to_numeric(df_clean)
df_clean = impute_missing_values(df_clean)
df_clean = create_vehicle_age_feature(df_clean)

Dropped 'NumberOfVehiclesInFleet' (0 non-nulls).
Cleaned and converted 'ExcessSelected' to numeric.
Cleaned and converted 'CapitalOutstanding' to numeric.
Imputed missing values in 'mmcode' with median: 60,058,415.00
Imputed missing values in 'Cylinders' with median: 4.00
Imputed missing values in 'cubiccapacity' with median: 2,694.00
Imputed missing values in 'kilowatts' with median: 111.00
Imputed missing values in 'NumberOfDoors' with median: 4.00
Imputed missing values in 'CustomValueEstimate' with 0.
Created 'VehicleAge' feature.


In [39]:


# ---  Final Robust Calculation of Variability Statistics ---

# Define the final list of numerical columns for analysis
numerical_features = [
    'RegistrationYear', 'Cylinders', 'cubiccapacity', 'kilowatts', 'NumberOfDoors',
    'CustomValueEstimate', 'CapitalOutstanding', 'SumInsured',
    'CalculatedPremiumPerTerm', 'ExcessSelected', 'TotalPremium',
    'TotalClaims', 'VehicleAge'
]

# Ensure only existing columns are included
numerical_features = [col for col in numerical_features if col in df_clean.columns]

# 1. Use .describe() for standard metrics
variability_stats_desc = df_clean[numerical_features].describe().T

# 2. Calculate Variance explicitly and add it
variance_series = df_clean[numerical_features].var()
variability_stats_desc['var'] = variance_series

# 3. Calculate the custom variability metrics (Range, IQR, CV)
variability_stats_desc['range'] = variability_stats_desc['max'] - variability_stats_desc['min']
variability_stats_desc['iqr'] = variability_stats_desc['75%'] - variability_stats_desc['25%']

# 4. Calculate CV (Coefficient of Variation)
variability_stats_desc['cv (%)'] = np.where(
    variability_stats_desc['mean'] != 0,
    (variability_stats_desc['std'] / variability_stats_desc['mean']) * 100,
    0
)

# 5. Select the final report columns (Now 'var' is guaranteed to be in the index)
variability_report = variability_stats_desc[[
    'count', 'mean', 'std', 'var', 'range', 'iqr', 'cv (%)'
]]

# Format the output for clarity
pd.options.display.float_format = '{:,.2f}'.format
print("\n## 📊 Variability Statistics Report for Numerical Features")
print(variability_report)


## 📊 Variability Statistics Report for Numerical Features
                                count           mean            std  \
RegistrationYear         1,000,098.00       2,010.23           3.26   
Cylinders                1,000,098.00           4.05           0.29   
cubiccapacity            1,000,098.00       2,466.87         442.71   
kilowatts                1,000,098.00          97.22          19.39   
NumberOfDoors            1,000,098.00           4.02           0.47   
CustomValueEstimate      1,000,098.00      49,714.82     281,048.10   
CapitalOutstanding       1,000,096.00      61,610.22     515,541.65   
SumInsured               1,000,098.00     604,172.73   1,508,331.84   
CalculatedPremiumPerTerm 1,000,098.00         117.88         399.70   
ExcessSelected             111,471.00 949,372,393.51 970,907,257.32   
TotalPremium             1,000,098.00          61.91         230.28   
TotalClaims              1,000,098.00          64.86       2,384.07   
VehicleAge        

In [40]:
df_clean[numerical_features].describe()

,RegistrationYear,Cylinders,cubiccapacity,kilowatts,NumberOfDoors,CustomValueEstimate,CapitalOutstanding,SumInsured,CalculatedPremiumPerTerm,ExcessSelected,TotalPremium,TotalClaims,VehicleAge
count,"1,000,098.00","1,000,098.00","1,000,098.00","1,000,098.00","1,000,098.00","1,000,098.00","1,000,096.00","1,000,098.00","1,000,098.00","111,471.00","1,000,098.00","1,000,098.00","1,000,098.00"
mean,"2,010.23",4.05,"2,466.87",97.22,4.02,"49,714.82","61,610.22","604,172.73",117.88,"949,372,393.51",61.91,64.86,4.53
std,3.26,0.29,442.71,19.39,0.47,"281,048.10","515,541.65","1,508,331.84",399.70,"970,907,257.32",230.28,"2,384.07",3.28
min,"1,987.00",0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.00,250.00,-782.58,"-12,002.41",0.00
25%,"2,008.00",4.00,"2,237.00",75.00,4.00,0.00,0.00,"5,000.00",3.22,"1,000,003,000.00",0.00,0.00,2.00
50%,"2,011.00",4.00,"2,694.00",111.00,4.00,0.00,0.00,"7,500.00",8.44,"1,000,005,000.00",2.18,0.00,4.00
75%,"2,013.00",4.00,"2,694.00",111.00,4.00,0.00,0.00,"250,000.00",90.00,"1,000,005,000.00",21.93,0.00,7.00
max,"2,015.00",10.00,"12,880.00",309.00,6.00,"26,550,000.00","28,570,000.00","12,636,200.00","74,422.17","10,000,010,000.00","65,282.60","393,092.11",28.00


#### ○ Data Summarization: